Import libraries

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import Trainer, TrainingArguments
from datasets import DatasetDict, load_dataset

Load Dataset and Model

In [22]:
dataset = load_dataset("json", data_files={
    "train": "mahasquad\\train.json",
    "validation": "mahasquad\\val.json",
    "test": "mahasquad\\test.json"
})

In [23]:
# Load the XLM-R tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-roberta")
model = AutoModelForQuestionAnswering.from_pretrained("l3cube-pune/marathi-roberta")

c:\Users\cl502_19\Desktop\Domain-LLM\LLMProj\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at l3cube-pune/marathi-roberta and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preprocessing function

In [24]:
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['data'],
        num_rows: 118516
    })
    validation: Dataset({
        features: ['data'],
        num_rows: 11873
    })
    test: Dataset({
        features: ['data'],
        num_rows: 11803
    })
})
{'data': {'answers': {'answer_start': [], 'text': []}, 'context': 'प्रेस्बिटेरियनिझम हा उत्तर आयर्लंडमधील सर्वात मोठा प्रोटेस्टंट संप्रदाय आहे आणि आयर्लंड बेटावर (आयर्लंडच्या अँग्लिकन चर्च नंतर) दुसरा सर्वात मोठा संप्रदाय आहे, [उद्धरण आवश्यक आहे] आणि स्कॉटिश वृक्षारोपण स्थायिकांनी अल्स्टरमध्ये आणले होते ज्यांना जेम्स व्ही द्वारे स्थलांतर करण्यास जोरदार प्रोत्साहन दिले होते. स्कॉटलंडचा, नंतर इंग्लंडचा जेम्स पहिला. अंदाजे १००,००० स्कॉटिश प्रेस्बिटेरियन्स १६०७ आणि १६९० मध्ये बॉयनच्या लढाईदरम्यान आयर्लंडच्या उत्तरेकडील काउंटीजमध्ये स्थलांतरित झाले. अल्स्टर आणि उर्वरित आयर्लंडमधील रोमन कॅथोलिकांसह प्रेस्बिटेरियन, १९व्या शतकाच्या सुरुवातीस ते मागे घेण्यापर्यंत भेदभाव करणाऱ्या दंड कायद्यांतर्गत त्रास सहन करावा लागला. आयर्लंडमधील प्रेस्

In [25]:
def preprocess_data(examples):
    # Access fields inside 'data'
    questions = [item["question"] for item in examples["data"]]
    contexts = [item["context"] for item in examples["data"]]
    answers = [item["answers"] for item in examples["data"]]
    
    # Tokenize the inputs with truncation and padding
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation=True,
        padding="max_length"
    )
    
    # Prepare start and end positions for the answers
    start_positions = [ans["answer_start"][0] if ans["answer_start"] else 0 for ans in answers]
    end_positions = [
        (ans["answer_start"][0] + len(ans["text"][0])) if ans["answer_start"] and ans["text"] else 0 
        for ans in answers
    ]
    
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Take subset of dataset

In [26]:
# Set a fraction of the dataset to use
fraction = 0.05  # Use 5% of the dataset

# Shuffle and select a subset of the training, validation, and test sets
train_subset = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * fraction)))
validation_subset = dataset["validation"].shuffle(seed=42).select(range(int(len(dataset["validation"]) * fraction)))
test_subset = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * fraction)))

# Create a new DatasetDict with the subsets
subset_dataset = DatasetDict({
    "train": train_subset,
    "validation": validation_subset,
    "test": test_subset
})

# Check the sizes of the subsets to confirm
print(f"Train subset size: {len(subset_dataset['train'])}")
print(f"Validation subset size: {len(subset_dataset['validation'])}")
print(f"Test subset size: {len(subset_dataset['test'])}")

Train subset size: 5925
Validation subset size: 593
Test subset size: 590


In [27]:
# Proceed with tokenization and training using subset_dataset
tokenized_subset = subset_dataset.map(preprocess_data, batched=True)







Map: 100%|██████████| 593/593 [00:00<00:00, 1626.18 examples/s]


Evaluation parameter

In [28]:
import evaluate

# Load evaluation metrics
metric = evaluate.load("squad_v2")

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    return metric.compute(predictions=predictions, references=references)

Training Arguments and Trainer

In [29]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./qa_subset_maharoberta_results",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [30]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_subset["train"],
    eval_dataset=tokenized_subset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Training the model

In [31]:
# Train the model
trainer.train()

 33%|███▎      | 741/2223 [47:50:39<95:41:18, 232.44s/it]

                                                  


                                      
  1%|▏         | 10/741 [02:05<2:30:00, 12.31s/it] 



{'loss': 5.9294, 'grad_norm': 34.9787712097168, 'learning_rate': 2.959514170040486e-05, 'epoch': 0.01}


                                                  


                                         
  3%|▎         | 20/741 [04:08<2:27:06, 12.24s/it]  



{'loss': 5.6452, 'grad_norm': 14.557489395141602, 'learning_rate': 2.9190283400809717e-05, 'epoch': 0.03}


                                                  


                                         
  4%|▍         | 30/741 [06:11<2:25:17, 12.26s/it]  



...

{'loss': 3.4159, 'grad_norm': 2.4425880908966064, 'learning_rate': 4.4534412955465585e-07, 'epoch': 0.99}


                                                   


                                         
100%|█████████▉| 740/741 [2:30:23<00:11, 11.61s/it] 



{'loss': 3.5965, 'grad_norm': 7.399904727935791, 'learning_rate': 4.048582995951417e-08, 'epoch': 1.0}


100%|██████████| 741/741 [2:30:32<00:00, 10.65s/it]












































































IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Saving the model

In [32]:
# Save the fine-tuned model
model.save_pretrained("marathi-qa-20-mahasquad")
tokenizer.save_pretrained("marathi-qa-20-mahasquad")

('marathi-qa-20-mahasquad\\tokenizer_config.json',
 'marathi-qa-20-mahasquad\\special_tokens_map.json',
 'marathi-qa-20-mahasquad\\sentencepiece.bpe.model',
 'marathi-qa-20-mahasquad\\added_tokens.json',
 'marathi-qa-20-mahasquad\\tokenizer.json')